In [4]:
import pandas as pd
from WindPy import w
from datetime import datetime


In [ ]:
w.start()

In [ ]:
w.wset()

In [ ]:
now = datetime.now().strftime('%Y%m%d')
etf_code = {'510500.SH','510300.SH','510050.SH','588000.SH','588080.SH'}
for code in etf_code:

    data_set = w.wset("optionchain", f"date={now};us_code={code};option_var=all;call_put=all;")
    basic_info = pd.DataFrame(data=data_set.Data,index=data_set.Fields).T   # 获取期权的基本信息
    
    data = w.w

    

In [5]:
import sys
sys.path.append('..')

import json 
import warnings
import pandas as pd
# import QUANTAXIS as QA
from QHData.query import query
from QHMlflow.secuntial import Sequential
from QHFactor.transformer import Load_X_y
from QHData.data_load import DataLoader,CodesLoader
from sklearn.model_selection import train_test_split

from QHMlflow.estimator import LGBMRollingRegressor
from QHMlflow.features_selector import LGBMFeatureSelector

warnings.filterwarnings('ignore')

In [39]:
# 1 查询数据

seq = Sequential(name='STK_UPLIMIT500_1D')  # 初始化交易框架

q = query()
# --------------------------------------------------------------------------------------
code_loader = CodesLoader()
code_hs300 = code_loader.load_stk_codes('hs300.json')
code_list = code_loader.load_stk_codes('zz500.json')      # 中证500成分股
code_qql = code_loader.load_stk_codes('qql.json')         # 敲敲乐报价股票

df_stocks = q.fetch_stock_list()    # 获取全市场股票信息
filter_codes = df_stocks[-df_stocks.name.str.startswith(('*',"ST"))]                    # 过滤掉*ST/ST的股票
codes_20cm = filter_codes[filter_codes.code.str.match(r'300|301|688|689')].code.tolist()      # 涨停板为20cm的股票
codes_10cm = filter_codes[-filter_codes.code.str.match(r'300|301|688|689')].code.tolist()     # 涨停板为10cm的股票
# --------------------------------------------------------------------------------------

data_loader = DataLoader(start_date='2024-02-01',end_date='2024-11-29',freq='daily')
stk_data = data_loader.load_stock(codes=codes_10cm)

# stk_data = stk_data.groupby(level=1).filter(lambda x:len(x) > 252)  # 过滤掉上市小于1年的股票

# # 2、计算因子
# factors = seq.calculate_factors(stk_data.fillna(0), window=60)    # 计算因子
# returns = stk_data.close.unstack().pct_change(periods=1).shift(-1).stack() # 预测未来一天的涨跌

[2024-11-30 20:52:11 - INFO] Model name: STK_UPLIMIT500_1D
[2024-11-30 20:52:11 - INFO] Training started at 2024-11-30 20:52:11


In [40]:
stk_data

open  close   high    low     volume       amount
date       code                                                      
2024-02-01 000001   8.60   8.60   8.66   8.53  1055259.0  993481920.0
           000002   9.53   9.44   9.66   9.42   627860.0  598048576.0
           000004  11.60  12.01  12.45  11.50    51659.0   62090536.0
           000006   4.13   3.93   4.19   3.75   554324.0  216924752.0
           000007   4.31   4.29   4.34   4.25    27204.0   11615702.0
...                  ...    ...    ...    ...        ...          ...
2024-11-29 605580   9.70   9.78   9.96   9.66    33325.0   32631060.0
           605588  46.21  47.97  48.50  46.20    29309.0  139854720.0
           605589  23.83  24.15  24.39  23.46   161423.0  387780352.0
           605598  18.70  20.64  20.64  18.61    69557.0  139941520.0
           605599  11.21  11.50  11.57  11.19    90081.0  103068672.0

[606861 rows x 6 columns]

In [57]:
recent_20_days = stk_data.groupby('code').tail(20)

# 计算涨停、跳空缺口和连续上涨的条件
recent_20_days['gap_up'] = recent_20_days['open'] > recent_20_days['close'].shift(1)
recent_20_days['up_days'] = recent_20_days['close'] > recent_20_days['close'].shift(1)


# 计算每个股票的条件
stock_conditions = recent_20_days.groupby('code').agg(
    has_limit_up=('close', lambda x: (x > x.shift(1) * 1.098).any()),
    has_gap_up=('gap_up', 'any'),
    consecutive_up_days=('up_days', lambda x: x.rolling(window=5).sum().max() >= 5),
    recent_5_days_volume_avg=('volume', lambda x: x.tail(5).mean())
).reset_index()


# 计算最近20天的成交均量
recent_20_days_volume_avg = recent_20_days.groupby('code')['volume'].mean().reset_index(name='recent_20_days_volume_avg')

# 合并条件和均量数据
stock_conditions = stock_conditions.merge(recent_20_days_volume_avg, on='code')

# 筛选符合条件的股票
selected_stocks = stock_conditions[
    (stock_conditions['has_limit_up']) &
    (stock_conditions['has_gap_up']) &
    (stock_conditions['consecutive_up_days']) &
    (stock_conditions['recent_5_days_volume_avg'] >= 2.5 * stock_conditions['recent_20_days_volume_avg'])
]

In [58]:
selected_stocks

,code,has_limit_up,has_gap_up,consecutive_up_days,recent_5_days_volume_avg,recent_20_days_volume_avg
248,000753,True,True,True,1466301.6,545503.70
385,000973,True,True,True,2124683.6,722934.75
580,002095,True,True,True,604696.6,218034.75
768,002301,True,True,True,624846.6,246107.35
1155,002741,True,True,True,874322.0,328366.00
1608,600249,True,True,True,643119.4,256290.70
1718,600405,True,True,True,1201618.0,459740.45
1882,600628,True,True,True,434250.0,167295.75
2096,600889,True,True,True,62803.2,17547.85
2942,603976,True,True,True,68424.6,26322.15
